In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [6]:
df=pd.read_csv('Toronto_Data.csv')

In [7]:
df.head()

,Unnamed: 0,Postal code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
Toronto_data = df[df['Borough'] == 'West Toronto'].reset_index(drop=True)
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
3,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325


In [29]:
### Getting the address of the West Toronto using geopy.geocoders---> Nominatim

address = 'West Toronto, Toronto'

geolocator = Nominatim(user_agent="West_Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of West Toronto are 43.6534817, -79.3839347.


In [35]:
# create map of West Toronto Borogh using latitude and longitude values

map_West_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_West_Toronto)  
    
map_West_Toronto


In [36]:
CLIENT_ID = 'MRFRYAWOIOGUEGY1S5UKMZHRN2WFOG3NUQFFXRCU2JLTH5WI' # your Foursquare ID
CLIENT_SECRET = '0OA02GTF3XJDKGZB1SEANA2HIL2FJN5LNMLZBHRLMVSBJXXH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MRFRYAWOIOGUEGY1S5UKMZHRN2WFOG3NUQFFXRCU2JLTH5WI
CLIENT_SECRET:0OA02GTF3XJDKGZB1SEANA2HIL2FJN5LNMLZBHRLMVSBJXXH


In [43]:
##  a function to gather and clean venues nearby all the neighborhoods in West Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:

West_Toronto_Venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )


Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , The Junction South
Parkdale , Roncesvalles
Runnymede , Swansea


In [55]:
print(West_Toronto_Venues.shape)
West_Toronto_Venues.head()

(155, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dufferin , Dovercourt Village",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dufferin , Dovercourt Village",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dufferin , Dovercourt Village",43.669005,-79.442259,Planet Fitness,43.667588,-79.442574,Gym / Fitness Center
3,"Dufferin , Dovercourt Village",43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store
4,"Dufferin , Dovercourt Village",43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery


In [56]:
West_Toronto_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
"Dufferin , Dovercourt Village",15,15,15,15,15,15
"High Park , The Junction South",23,23,23,23,23,23
"Little Portugal , Trinity",41,41,41,41,41,41
"Parkdale , Roncesvalles",13,13,13,13,13,13
"Runnymede , Swansea",40,40,40,40,40,40


In [57]:
print('There are {} uniques categories.'.format(len(West_Toronto_Venues['Venue Category'].unique())))

There are 81 uniques categories.


In [59]:
## ANALYSING ALL THE POSTAL GROUPED NEIGHBORHOODS


# one hot encoding
West_Toronto_onehot = pd.get_dummies(West_Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
West_Toronto_onehot['Neighborhood'] = West_Toronto_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [West_Toronto_onehot.columns[-1]] + list(West_Toronto_onehot.columns[:-1])
West_Toronto_onehot = West_Toronto_onehot[fixed_columns]

West_Toronto_onehot.head()

,Neighborhood,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,IT Services,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Post Office,Pub,Record Shop,Restaurant,Sandwich Place,Speakeasy,Stadium,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Dufferin , Dovercourt Village",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dufferin , Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dufferin , Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dufferin , Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Dufferin , Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
## GROUPING by all the Neighbourhoods

West_Toronto_Grouped=West_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
West_Toronto_Grouped

,Neighborhood,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,IT Services,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Post Office,Pub,Record Shop,Restaurant,Sandwich Place,Speakeasy,Stadium,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Brockton , Parkdale Village , Exhibition Place",0.00000,0.000000,0.00000,0.000000,0.043478,0.000000,0.043478,0.00000,0.000000,0.00000,0.086957,0.000000,0.043478,0.130435,0.000000,0.043478,0.00000,0.086957,0.043478,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000,0.00000,0.000000,0.043478,0.000000,0.000000,0.000,0.00000,0.043478,0.043478,0.000000,0.000,0.000,0.00000,0.000,0.043478,0.043478,0.00000,0.00000,0.00000,0.000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.086957,0.000000,0.043478,0.043478,0.000000,0.00000,0.000,0.00,0.00000,0.043478,0.000,0.000000,0.043478,0.000000,0.00,0.000,0.000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
1,"Dufferin , Dovercourt Village",0.00000,0.000000,0.00000,0.066667,0.133333,0.066667,0.066667,0.00000,0.000000,0.00000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.00000,0.066667,0.000000,0.066667,0.000,0.000,0.00000,0.000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000,0.00000,0.000000,0.066667,0.00000,0.000000,0.066667,0.00000,0.000000,0.066667,0.000000,0.000000,0.133333,0.00000,0.000,0.00,0.00000,0.000000,0.000,0.000000,0.000000,0.066667,0.00,0.000,0.000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
2,"High Park , The Junction South",0.00000,0.086957,0.00000,0.000000,0.043478,0.000000,0.043478,0.00000,0.043478,0.00000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.043478,0.000000,0.000000,0.000,0.043478,0.000,0.043478,0.000,0.00000,0.043478,0.043478,0.043478,0.000000,0.000,0.00000,0.043478,0.000000,0.000000,0.000,0.000,0.00000,0.000,0.000000,0.043478,0.00000,0.00000,0.00000,0.000,0.00000,0.086957,0.000000,0.00000,0.000000,0.043478,0.00000,0.000000,0.043478,0.000000,0.000000,0.000000,0.00000,0.000,0.00,0.00000,0.000000,0.000,0.043478,0.000000,0.000000,0.00,0.000,0.000,0.086957,0.00000,0.00000,0.00000,0.00000,0.00000
3,"Little Portugal , Trinity",0.02439,0.000000,0.04878,0.000000,0.000000,0.000000,0.097561,0.02439,0.000000,0.02439,0.000000,0.024390,0.000000,0.048780,0.000000,0.000000,0.02439,0.024390,0.000000,0.000,0.024390,0.02439,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000,0.02439,0.000000,0.000000,0.000000,0.024390,0.000,0.02439,0.000000,0.000000,0.000000,0.000,0.000,0.02439,0.000,0.000000,0.024390,0.02439,0.02439,0.02439,0.000,0.04878,0.000000,0.000000,0.02439,0.000000,0.000000,0.02439,0.000000,0.024390,0.000000,0.000000,0.000000,0.02439,0.000,0.00,0.02439,0.073171,0.000,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.02439,0.04878,0.04878,0.02439,0.02439
4,"Parkdale , Roncesvalles",0.00000,0.000000,0.0

In [73]:
#Printing Each neighborhood with top 5 venues

num_top_venues = 5

for hood in West_Toronto_Grouped['Neighborhood']:
    
    print("----"+hood+"----")
    
    # Now we Transpose the WEST_Toronto_Grouped matrix at a particular Neighborhood
    
    temp = West_Toronto_Grouped[West_Toronto_Grouped['Neighborhood'] == hood].T.reset_index() 
    temp.columns = ['venue','freq'] # addind collumn names
    
    #Removing the first row now cuz we have our own coumn names
    temp = temp.iloc[1:]
    
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2}) # rounding up the freq value

    #Sorting the Table and dispayinf top 5 venues
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)) 
    print('\n')


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Café  0.13
1           Nightclub  0.09
2         Coffee Shop  0.09
3      Breakfast Spot  0.09
4  Italian Restaurant  0.04


----Dufferin , Dovercourt Village----
           venue  freq
0       Pharmacy  0.13
1         Bakery  0.13
2  Grocery Store  0.07
3           Park  0.07
4    Music Venue  0.07


----High Park , The Junction South----
                 venue  freq
0   Mexican Restaurant  0.09
1  Arts & Crafts Store  0.09
2      Thai Restaurant  0.09
3                 Café  0.09
4                Diner  0.04


----Little Portugal , Trinity----
                   venue  freq
0                    Bar  0.10
1             Restaurant  0.07
2            Men's Store  0.05
3       Asian Restaurant  0.05
4  Vietnamese Restaurant  0.05


----Parkdale , Roncesvalles----
                         venue  freq
0                    Gift Shop  0.15
1                 Dessert Shop  0.08
2               B

In [72]:
# THIS is a function definition for sorting values in descending order

def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = West_Toronto_Grouped['Neighborhood']

for ind in np.arange(West_Toronto_Grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(West_Toronto_Grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton , Parkdale Village , Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Italian Restaurant,Pet Store,Climbing Gym,Intersection,Burrito Place,Restaurant
1,"Dufferin , Dovercourt Village",Pharmacy,Bakery,Gym / Fitness Center,Supermarket,Middle Eastern Restaurant,Music Venue,Café,Brewery,Park,Grocery Store
2,"High Park , The Junction South",Mexican Restaurant,Arts & Crafts Store,Thai Restaurant,Café,Speakeasy,Fast Food Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Flea Market,Music Venue
3,"Little Portugal , Trinity",Bar,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Gift Shop,Korean Restaurant,Juice Bar
4,"Parkdale , Roncesvalles",Gift Shop,Movie Theater,Cuban Restaurant,Dog Run,Coffee Shop,Bar,Eastern European Restaurant,Bookstore,Italian Restaurant,Breakfast Spot


In [88]:
## K-Means Clusters 


# set number of clusters
kclusters = 5

West_Toronto_grouped_clustering = West_Toronto_Grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(West_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 1, 0, 1], dtype=int32)

In [89]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

West_Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
West_Toronto_merged = West_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

West_Toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,2,Pharmacy,Bakery,Gym / Fitness Center,Supermarket,Middle Eastern Restaurant,Music Venue,Café,Brewery,Park,Grocery Store
1,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750,1,Bar,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Gift Shop,Korean Restaurant,Juice Bar
2,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,3,Café,Nightclub,Breakfast Spot,Coffee Shop,Italian Restaurant,Pet Store,Climbing Gym,Intersection,Burrito Place,Restaurant
3,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763,4,Mexican Restaurant,Arts & Crafts Store,Thai Restaurant,Café,Speakeasy,Fast Food Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Flea Market,Music Venue
4,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325,0,Gift Shop,Movie Theater,Cuban Restaurant,Dog Run,Coffee Shop,Bar,Eastern European Restaurant,Bookstore,Italian Restaurant,Breakfast Spot


In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(West_Toronto_merged['Latitude'], West_Toronto_merged['Longitude'], West_Toronto_merged['Neighborhood'], West_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [97]:
## Examine Clusters by Table


West_Toronto_merged.loc[West_Toronto_merged['Cluster Labels'] == 0, West_Toronto_merged.columns[[2] + list(range(5, West_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Parkdale , Roncesvalles",0,Gift Shop,Movie Theater,Cuban Restaurant,Dog Run,Coffee Shop,Bar,Eastern European Restaurant,Bookstore,Italian Restaurant,Breakfast Spot


In [98]:
West_Toronto_merged.loc[West_Toronto_merged['Cluster Labels'] == 1, West_Toronto_merged.columns[[2] + list(range(5, West_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Little Portugal , Trinity",1,Bar,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Gift Shop,Korean Restaurant,Juice Bar
5,"Runnymede , Swansea",1,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Pub,Italian Restaurant,Tennis Court,Tea Room,Indie Movie Theater,Juice Bar


In [99]:
West_Toronto_merged.loc[West_Toronto_merged['Cluster Labels'] == 2, West_Toronto_merged.columns[[2] + list(range(5, West_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Dufferin , Dovercourt Village",2,Pharmacy,Bakery,Gym / Fitness Center,Supermarket,Middle Eastern Restaurant,Music Venue,Café,Brewery,Park,Grocery Store


In [100]:
West_Toronto_merged.loc[West_Toronto_merged['Cluster Labels'] == 3, West_Toronto_merged.columns[[2] + list(range(5, West_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Brockton , Parkdale Village , Exhibition Place",3,Café,Nightclub,Breakfast Spot,Coffee Shop,Italian Restaurant,Pet Store,Climbing Gym,Intersection,Burrito Place,Restaurant


In [101]:
West_Toronto_merged.loc[West_Toronto_merged['Cluster Labels'] == 4, West_Toronto_merged.columns[[2] + list(range(5, West_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"High Park , The Junction South",4,Mexican Restaurant,Arts & Crafts Store,Thai Restaurant,Café,Speakeasy,Fast Food Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Flea Market,Music Venue
